<a href="https://colab.research.google.com/github/jasmehar-k/breast-cancer-prediction/blob/main/breast_cancer_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn #neural networks
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
data = load_breast_cancer()
x = data.data
y = data.target


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state =42)


In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
x_train = torch.tensor(x_train, dtype=torch.float32).to(device)
x_test = torch.tensor(x_test, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

In [ ]:
class NeuralNetwork(nn.Module):

  def __init__(self, input_size, hidden_size, output_size):
    super(NeuralNetwork, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, output_size)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
      out = self.fc1(x)
      out = self.relu(out)
      out = self.fc2(out)
      out = self.sigmoid(out)
      return out

In [ ]:
input_size = x_train.shape[1]
hidden_size = 64
output_size = 1
learning_rate = 0.001
num_epochs = 100

In [ ]:
model = NeuralNetwork(input_size, hidden_size, output_size).to(device)

In [ ]:
criteria = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
for epoch in range(num_epochs):
  model.train()
  optimizer.zero_grad()
  outputs = model(x_train)
  loss = criteria(outputs, y_train.view(-1,1))
  loss.backward()
  optimizer.step()

  with torch.no_grad():
    predicted = outputs.round()
    correct = (predicted == y_train.view(-1,1)).float().sum()
    accuracy = correct/y_train.size(0)

  if (epoch + 1)% 10 == 0:
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy.item()*100:.2f}%")

Epoch [10/100], Loss: 0.5803, Accuracy: 85.93%
Epoch [20/100], Loss: 0.4617, Accuracy: 91.65%
Epoch [30/100], Loss: 0.3651, Accuracy: 93.63%
Epoch [40/100], Loss: 0.2897, Accuracy: 94.29%
Epoch [50/100], Loss: 0.2335, Accuracy: 94.73%
Epoch [60/100], Loss: 0.1933, Accuracy: 96.04%
Epoch [70/100], Loss: 0.1647, Accuracy: 96.70%
Epoch [80/100], Loss: 0.1441, Accuracy: 96.70%
Epoch [90/100], Loss: 0.1286, Accuracy: 96.92%
Epoch [100/100], Loss: 0.1166, Accuracy: 97.80%


In [ ]:
model.eval()
with torch.no_grad():
  outputs = model(x_train)
  predicted = outputs.round()
  correct = (predicted == y_train.view(-1,1)).float().sum()
  accuracy = correct/y_train.size(0)
  print(f"Accuracy on training data: {accuracy.item()*100:.2f}")


Accuracy on training data: 97.80


In [ ]:
model.eval()
with torch.no_grad():
  outputs = model(x_test)
  predicted = outputs.round()
  correct = (predicted == y_test.view(-1,1)).float().sum()
  accuracy = correct/y_test.size(0)
  print(f"Accuracy on test data: {accuracy.item()*100:.2f}")


Accuracy on test data: 97.37
